In [12]:
# !pip install optuna
import optuna
import optuna.integration.lightgbm as lgb
import os
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
# Scikit-learn
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV
 
import joblib
import operator
# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
# from matplotlib_venn import venn2, venn3
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter

# 他ファイルのインポート・リロード
import imp
import yfinance as yf



def light_gbm():
      #全時系列データの呼び出し
    model_stock = 'sandp500'
    path = '../../../stock_data/stock_code/{}.csv'.format(model_stock)
    path = '/Users/suzukiryotaro/Desktop/python_stock/stock_data/stock_code/sandp500.csv'
    data = pd.read_csv(path)
    print(data)

    # 作業ディレクトリの移動と確認
    os.chdir('/Users/suzukiryotaro/Desktop/python_stock/machine_learning/XGBboost/model1/model')
    !ls
    # 他ファイルのインポート・リロード
    import data_process
    imp.reload(data_process)
    
    
    preprocess_data = pd.DataFrame()
    count = 0
    for  file in data['symbol'] :
        count = count +1
        print(count)
        print(file)
        try:
            df = yf.download(file, start='2014-01-01',end='2020-01-01')
            df = data_process.create_data(df)
#             print(df)
            preprocess_data = preprocess_data.append(df)

        except: 
            print('errror')
            continue

    save_path = '/Users/suzukiryotaro/Desktop/python_stock/machine_learning/ansamble/lightgbm/preprocss_data.csv'
#     preprocess_data.to_csv(save_path)

    df = pd.read_csv(save_path, index_col=0)
    
#     print(df)
    #選択する特徴量
    cols =  ['return','sma210'

            ,'ratio_sma75_30', 'ratio_sma75_150', 'ratio_sma105_30', 
            'ratio_sma105_150', 'ratio_sma135_150'
            ,'Highest161','adosc', 'sma10', 
          'adosc-SG' ,'typical-price',"ema270"
       ,'Highest121','Highest45,81days_ago',

    'today_by_sma30ratio','today_by_sma45ratio','today_by_sma60ratio','today_by_sma120ratio'


           ]
    df = df[cols]
    df = df.set_axis(["a","b","c","d","e","f","g",'h','i','j','k','l','m','n','o','p','q','r','s'], axis=1)
#     print(df)

    X = df.copy()
    y = X.pop('a')
    X.reindex(columns=["a","b","c","d","e","f","g",'h','i','j','k','l','m','n','o','p','q','r'])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=11)
    # train用・valid用のデータセットを生成する
    train = lgb.Dataset(X_train, y_train)
    val = lgb.Dataset(X_val, y_val)
    
    
    
    # ハイパーパラメータサーチ&モデル構築
    params = {'objective': 'regression',
              'metric': 'rmse',
              'random_seed':0} 

    gbm_o = lgb.train(params,
                        train,
                        valid_sets=val,
                        early_stopping_rounds=100,
                        verbose_eval=200,)

    # 調整後モデルで予測の実行
    y_trainval_pred = gbm_o.predict(X_train,num_iteration=gbm_o.best_iteration)
    y_test_pred = gbm_o.predict(X_val,num_iteration=gbm_o.best_iteration)

    # ベストパラメータの取得
    best_params = gbm_o.params
    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))

    # 評価スコアの計算
    r2_trainval = r2_score(y_train, y_trainval_pred)
    r2_test = r2_score(y_val, y_test_pred)

    print("r2_train:{0:.4}".format(r2_trainval))
    print("r2_test:{0:.4}".format(r2_test))

    
light_gbm()
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
#     # ベストなパラメータ、途中経過を保存する
#     best_params, tuning_history = dict(), list() 

#     # Model
#     model = lgb.LGBMRegressor()

#     lgbm_params = {'objective': 'regression', 
#                'learning_rate':0.01, 
#                'num_boost_round':10000, 
#                'early_stopping_rounds':10
#               }

#     # 学習
#     model = lgb.train(lgbm_params, 
#                       lgb_train, 
#                       valid_sets=lgb_val, 
#                       verbose_eval=0,
#                       early_stopping_rounds=100
#                      ) 
#     best.params
#     best.best_iteration
#     best.best_score

# light_gbm()
#     #  print(best_params)
#     #  print(tuning_history)
#     # # パラメーター設定
#     # light_params = {"max_depth": 20,
#     #                 'task': 'train', # レーニング ⇔　予測predict
#     #                 'boosting_type': 'gbdt', # 勾配ブースティング
#     #                 'objective': 'reg:squarederror', # 目的関数：多値分類、マルチクラス分類 
#     #                 'metric': "rmse", # 検証用データセットで、分類モデルの性能を測る指標 
#     #                 'num_class': 2, # 目的変数のクラス数 
#     #                 'num_leaves': 23, # 決定木の複雑度を調整（初期値31）
#     #                 'min_data_in_leaf': 15, # データの最小数（初期値20） 
#     #                 "learning_rate": 0.025400643228660445,
#     #                 'gamma':0.20387750446308275, 
#     #               } 



#     # # fit
#     # model.fit(params,
#     #     X_train, y_train, 
#     #     eval_set=[(X_valid, y_valid)], 
#     #     verbose=100, 
#     #     num_boost_round=100,
#     #     best_params=best_params,
#     #     early_stopping_rounds=300,
#     # )

#     # model = lgb.train( trains, valid_sets=valids,
#     #                   verbose_eval=False,

#     #                   early_stopping_rounds=5,

#     #                   tuning_history=history)
#     # joblib.dump(model, '/content/drive/MyDrive/machine_learning/ansamble/model/lightgbm.joblib') 


feature_fraction, val_score: inf:   0%|          | 0/7 [11:56:55<?, ?it/s]


    symbol                 Name                  Sector
0      MMM           3M Company             Industrials
1      AOS      A.O. Smith Corp             Industrials
2      ABT  Abbott Laboratories             Health Care
3     ABBV          AbbVie Inc.             Health Care
4     ABMD              Abiomed             Health Care
..     ...                  ...                     ...
500    YUM      Yum! Brands Inc  Consumer Discretionary
501   ZBRA   Zebra Technologies  Information Technology
502    ZBH        Zimmer Biomet             Health Care
503   ZION        Zions Bancorp              Financials
504    ZTS               Zoetis             Health Care

[505 rows x 3 columns]


[I 2021-06-29 11:57:55,635] A new study created in memory with name: no-name-5c361509-fa3e-4355-bd91-1b8705a5f87c

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019756 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4339
[LightGBM] [Info] Number of data points in the train set: 520901, number of used features: 18
[LightGBM] [Info] Start training from score 0.023188
Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 0.357379
[400]	valid_0's rmse: 0.353001
[600]	valid_0's rmse: 0.349459
[800]	valid_0's rmse: 0.346406



feature_fraction, val_score: 0.343767:  14%|#4        | 1/7 [00:20<02:05, 20.89s/it][I 2021-06-29 11:58:16,700] Trial 0 finished with value: 0.3437665680366974 and parameters: {'feature_fraction': 0.7}. Best is trial 0 with value: 0.3437665680366974.

feature_fraction, val_score: 0.343767:  14%|#4        | 1/7 [00:20<02:05, 20.89s/it]

...


min_data_in_leaf, val_score: 0.298440:  80%|########  | 4/5 [04:58<01:16, 76.74s/it][I 2021-06-29 12:57:17,900] Trial 63 finished with value: 0.3034529698245187 and parameters: {'min_child_samples': 100}. Best is trial 62 with value: 0.2991363512168902.

min_data_in_leaf, val_score: 0.298440:  80%|########  | 4/5 [04:58<01:16, 76.74s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006858 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4339
[LightGBM] [Info] Number of data points in the train set: 520901, number of used features: 18
[LightGBM] [Info] Start training from score 0.023188
Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 0.333573
[400]	valid_0's rmse: 0.320515
[600]	valid_0's rmse: 0.311475
[800]	valid_0's rmse: 0.303706
[1000]	valid_0's rmse: 0.298169
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 0.298169



min_data_in_leaf, val_score: 0.298169: 100%|##########| 5/5 [06:09<00:00, 73.98s/it]


  Params: 
    objective: regression
    metric: rmse
    random_seed: 0
    feature_pre_filter: False
    lambda_l1: 0.5263404867949211
    lambda_l2: 0.006584498723266508
    num_leaves: 252
    feature_fraction: 1.0
    bagging_fraction: 0.7850836052089633
    bagging_freq: 3
    min_child_samples: 5
    num_iterations: 1000
    early_stopping_round: 100


NameError: name 'r2_score' is not defined